<a href="https://colab.research.google.com/github/joshuaghannan/ECEC247_Project/blob/jonathan-tests/colab_version_clean_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### loading modules and data

In [1]:
########################################################

# If running with Google Colab

from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
# # If using colab
 from google.colab import files
 files.upload()
# # select the 3 .py files (models, utils, data_utils)

Saving data_utils.py to data_utils.py
Saving models.py to models.py
Saving utils.py to utils.py


{'data_utils.py': b'import numpy as np\nimport scipy.signal as sig\nimport pywt\nfrom torch.utils.data import Dataset, DataLoader\nimport torch\n\n\'\'\'\nThe customized dataset and data augmentations are defined here.\n\n\'\'\'\nclass EEG_Dataset(Dataset):\n    \'\'\'\n    use use fold_idx to instantiate different train val datasets for k-fold cross validation\n\n    \'\'\'\n    def __init__ (self, X_train=None, y_train=None, p_train=None, X_val=None, y_val=None, p_val=None, X_test=None, y_test=None, p_test=None, mode=\'train\'):\n        if mode == \'train\':\n            self.X = X_train\n            self.y = y_train- 769\n            self.p = p_train\n            \n        elif mode == \'val\':\n            self.X = X_val\n            self.y = y_val- 769\n            self.p = p_val\n\n        elif mode == \'test\':\n            self.X = X_test\n            self.y = y_test - 769        \n            self.p = p_test\n\n    def __len__(self):\n        return (self.X.shape[0])\n    \n 

In [5]:
########################################################

# If running with Google Colab
# Create a folder "C247" and then store the project datasets within that folder
# Check that your datasets are setup correctly

!ls "/content/gdrive/My Drive/C247" # File path

data_utils.py	   person_test.npy	   X_test.npy	      y_train_valid.npy
EEG_loading.ipynb  person_train_valid.npy  X_train_valid.npy
models.py	   utils.py		   y_test.npy


In [3]:
from models import *
from utils import *
from data_utils import *

GPU is available


In [4]:
# X_test = np.load("X_test.npy")
# y_test = np.load("y_test.npy")
# person_train_valid = np.load("person_train_valid.npy")
# X_train_valid = np.load("X_train_valid.npy")
# y_train_valid = np.load("y_train_valid.npy")
# person_test = np.load("person_test.npy")

# Change if your directory is different

# dataset_path = './data/' # Yiming Path
dataset_path = "/content/gdrive/My Drive/C247/" 

X_test = np.load(dataset_path + "X_test.npy")
y_test = np.load(dataset_path + "y_test.npy")
person_train_valid = np.load(dataset_path + "person_train_valid.npy")
X_train_valid = np.load(dataset_path + "X_train_valid.npy")
y_train_valid = np.load(dataset_path + "y_train_valid.npy")
person_test = np.load(dataset_path + "person_test.npy")
print ('Training/Valid data shape: {}'.format(X_train_valid.shape))
print ('Test data shape: {}'.format(X_test.shape))
print ('Training/Valid target shape: {}'.format(y_train_valid.shape))
print ('Test target shape: {}'.format(y_test.shape))
print ('Person train/valid shape: {}'.format(person_train_valid.shape))
print ('Person test shape: {}'.format(person_test.shape))

train_fold, val_fold = Train_Val_Data(X_train_valid, y_train_valid)


Training/Valid data shape: (2115, 22, 1000)
Test data shape: (443, 22, 1000)
Training/Valid target shape: (2115,)
Test target shape: (443,)
Person train/valid shape: (2115, 1)
Person test shape: (443, 1)


### K-Fold Training and Validation (use k=1 to get results faster)

Data $\rightarrow$ Window $\rightarrow$ CWT

In [0]:
# setting up the data augmentation here
aug_type = 'cwt'

#WINDOW PARAMS
window_size = 200
window_stride = 20
vote_num = 20

#STFT PARAMS
stft_size = 
stft_stride = 5

#CWT PARAMS
cwt_level = 10
cwt_scale = 3

#RNN PARAMS
rnn_type = 'GRU'
dropout_param = 0.4
num_epochs = 20
hidden_size = 50
lr = 1e-3
weight_decay = 1e-3



for k in range(1):
    # indicate hyperparameters here
    print ('fold {}'.format(k+1))
    X_train, y_train, p_train = X_train_valid[train_fold[k]], y_train_valid[train_fold[k]], person_train_valid[train_fold[k]]
    X_val, y_val, p_val = X_train_valid[val_fold[k]], y_train_valid[val_fold[k]], person_train_valid[val_fold[k]]

    #apply window
    _, X_train, y_train, p_train = window_data(X_train,y_train,p_train,window_size=window_size,stride=window_stride)
    _, X_val, y_val, p_val = window_data(X_val,y_val,p_val,window_size=window_size,stride=window_stride)
    print('Data windowed...')

    #make data N(0,1)
    mu = np.mean(X_train, axis=(0,2), keepdims = True) 
    std = np.std(X_train, axis=(0,2), keepdims = True)
    X_train = standardize(X_train, mu, std)
    X_val = standardize(X_val, mu, std)
    
    #augment data with requested augmentation
    X_train, y_train, p_train = Aug_Data(X_train, y_train, p_train, aug_type=aug_type, window_size=window_size,cwt_level=cwt_level,cwt_scale=cwt_scale,stft_size=stft_size,stft_stride=stft_stride,window_stride=window_stride)
    X_val, y_val, p_val = Aug_Data(X_val, y_val, p_val, window_size=window_size, aug_type=aug_type, cwt_level=cwt_level,cwt_scale=cwt_scale,stft_size=stft_size,stft_stride=stft_stride,window_stride=window_stride)
    print('Transform applied...')
    model, criterion, optimizer = InitRNN(rnn_type=rnn_type, dropout=dropout_param, input_size=X_train.shape[1],hidden_size=hidden_size, lr=lr,weight_decay=weight_decay)
    EEG_trainset = EEG_Dataset(X_train=X_train, y_train=y_train, p_train=p_train, mode='train')
    EEG_trainloader = DataLoader(EEG_trainset, batch_size=128, shuffle=True)
    EEG_valset = EEG_Dataset(X_val=X_val, y_val=y_val, p_val=p_val, mode='val')
    EEG_valloader = DataLoader(EEG_valset, batch_size=128, shuffle=False)
    print('Data prepared, model initialized, beginning training...')
    best_model = TrainValRNN(model, criterion, optimizer, EEG_trainloader, EEG_valloader, num_epochs=num_epochs, aug_type='window', window_size=window_size, vote_num=vote_num)


fold 1
Data windowed...


### Training and Testing

In [0]:
TestRNN(model, X_test, y_test, person_test, aug_type=aug_type, window_size=window_size, vote_num=vote_num)

Testing Accuracy: 0.4718


In [0]:
# test

In [0]:
# test1

In [0]:
# test3